Adatok előfeldolgozása

In [81]:
import zipfile

zip_path = "legaltextdecoder.zip"
extract_path = "data"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


In [82]:
base_dir = "data"
all_data = []

json_file_count = 0
record_count = 0

for root, dirs, files in os.walk(base_dir):
    dirs[:] = [d for d in dirs if d != "consensus"]

    for file in files:
        if file.endswith(".json"):
            json_file_count += 1
            file_path = os.path.join(root, file)

            with open(file_path, "r", encoding="utf-8") as f:
                data = json.load(f)

                if isinstance(data, list):
                    all_data.extend(data)
                    record_count += len(data)
                else:
                    all_data.append(data)
                    record_count += 1

print(f"JSON files loaded: {json_file_count}")
print(f"Total data records: {record_count}")

JSON files loaded: 34
Total data records: 3897


In [83]:
texts = []
labels = []

for item in all_data:
    text = item.get("data", {}).get("text")

    if not text:
        continue

    annotations = item.get("annotations", [])
    if not annotations:
        continue

    ann = annotations[0]
    results = ann.get("result", [])
    if not results:
        continue

    choices = results[0].get("value", {}).get("choices", [])
    if not choices:
        continue

    label = choices[0]

    texts.append(text)
    labels.append(label)

print("Loaded data records:", len(texts))

Loaded data records: 3747


In [84]:
from collections import Counter

label_counts = Counter(labels)

print("Label distribution:")
for label, count in label_counts.items():
    print(f"{label}: {count}")


Label distribution:
5-Könnyen érthető: 1159
4-Érthető: 1122
3-Többé/kevésbé megértem: 816
2-Nehezen érthető: 450
1-Nagyon nehezen érthető: 200


In [13]:
#Baseline modell
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

#Túltanítás
train_texts = texts[:8]
train_labels = labels[:8]

test_texts = texts[8:]
test_labels = labels[8:]

label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_train = label_encoder.transform(train_labels)
y_test = label_encoder.transform(test_labels)

NUM_CLASSES = 5
y_train = np.clip(y_train, 0, NUM_CLASSES - 1)
y_test = np.clip(y_test, 0, NUM_CLASSES - 1)

y_train = to_categorical(y_train, num_classes=NUM_CLASSES)

MAX_FEATURES = 2000

tfidf = TfidfVectorizer(
    max_features=MAX_FEATURES,
    ngram_range=(1, 2),
    min_df=1
)

X_train = tfidf.fit_transform(train_texts).toarray()
X_test = tfidf.transform(test_texts).toarray()

print("TF-IDF feature size:", X_train.shape[1])

# -----------------------------
# Neural Network (MLP)
# -----------------------------
model = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(NUM_CLASSES, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# -----------------------------
# Train
# -----------------------------
model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=2,
    verbose=1
)


TF-IDF feature size: 86


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                │ (None, 32)             │         2,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,005 (15.64 KB)

 Trainable params: 4,005 (15.64 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.3167 - loss: 1.5985
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.2333 - loss: 1.5454    
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9500 - loss: 1.4953 
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7667 - loss: 1.4741 
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9500 - loss: 1.3850 
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8667 - loss: 1.3597
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8667 - loss: 1.3214 
Epoch 8/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9500 - loss: 1.2699 
Epoch 9/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8667 - loss: 1.1997 
Epoch 10/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7667 - loss: 1.2131
Epoch 11/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9500 - loss: 1.0694 
Epoch 12/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9500 

Az overfitt megtörtént.

In [85]:
import re

def clean_text(text):
    text = text.lower()                      # lowercase
    text = re.sub(r'\s+', ' ', text)        # collapse multiple spaces
    #text = re.sub(r'[^a-z0-9 ]', '', text)  # remove punctuation except letters/numbers
    return text.strip()

clean_texts = [clean_text(line) for line in texts]

Baseline modell betanítása

In [86]:
from sklearn.model_selection import train_test_split

# Train vs temp (val + test)
X_train, X_temp, y_train, y_temp = train_test_split(
    texts,
    labels,
    test_size=0.25,
    stratify=labels,
    random_state=42
)

# Val vs test
X_val, X_test, y_val, y_test = train_test_split(
    X_temp,
    y_temp,
    test_size=0.40,
    stratify=y_temp,
    random_state=42
)

print("Train size:", len(X_train))
print("Val size:  ", len(X_val))
print("Test size: ", len(X_test))


Train size: 2810
Val size:   562
Test size:  375


In [30]:
label_encoder = LabelEncoder()
label_encoder.fit(labels)  # Fit on all labels

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

# One-hot encoding for MLP
NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# TF-IDF Vectorization
# -----------------------------
MAX_FEATURES = 2000

tfidf = TfidfVectorizer(
    max_features=MAX_FEATURES,
    ngram_range=(1, 2),
    min_df=1
)

X_train_vec = tfidf.fit_transform(X_train).toarray()
X_val_vec   = tfidf.transform(X_val).toarray()
X_test_vec  = tfidf.transform(X_test).toarray()

print("TF-IDF feature size:", X_train_vec.shape[1])

# -----------------------------
# Neural Network (MLP)
# -----------------------------
model = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_vec.shape[1],)),
    Dense(32, activation='relu'),
    Dense(NUM_CLASSES, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# -----------------------------
# Train (with validation)
# -----------------------------
model.fit(
    X_train_vec,
    y_train_enc,
    validation_data=(X_val_vec, y_val_enc),
    epochs=20,
    batch_size=16,
    verbose=1
)

# -----------------------------
# Evaluate on test set
# -----------------------------
test_preds = model.predict(X_test_vec)
test_preds = np.argmax(test_preds, axis=1)
decoded_preds = label_encoder.inverse_transform(test_preds)

print("\nTest Classification Report:")
print(classification_report(y_test, decoded_preds))

TF-IDF feature size: 2000


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_39 (Dense)                │ (None, 32)             │        64,032 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_40 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_41 (Dense)                │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 65,253 (254.89 KB)

 Trainable params: 65,253 (254.89 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.3108 - loss: 1.5360 - val_accuracy: 0.3665 - val_loss: 1.4373
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4571 - loss: 1.3409 - val_accuracy: 0.4217 - val_loss: 1.3399
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6022 - loss: 1.0484 - val_accuracy: 0.4235 - val_loss: 1.3679
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7073 - loss: 0.8042 - val_accuracy: 0.4253 - val_loss: 1.4628
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7993 - loss: 0.6282 - val_accuracy: 0.4199 - val_loss: 1.5941
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8539 - loss: 0.4938 - val_accuracy: 0.4342 - val_loss: 1.7781
Epoch 7/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9010 - loss: 0.3580 - val_accuracy: 0.4164 - val_loss: 1.9300
Epoch 8/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9263 - loss: 0.2945 - val_accuracy: 0.

Early stopping hozzáadása


In [31]:
from tensorflow.keras.callbacks import EarlyStopping


# -----------------------------
# Encode labels
# -----------------------------
label_encoder = LabelEncoder()
label_encoder.fit(labels)  # Fit on all labels

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# TF-IDF Vectorization
# -----------------------------
MAX_FEATURES = 2000

tfidf = TfidfVectorizer(
    max_features=MAX_FEATURES,
    ngram_range=(1, 2),
    min_df=1
)

X_train_vec = tfidf.fit_transform(X_train).toarray()
X_val_vec   = tfidf.transform(X_val).toarray()
X_test_vec  = tfidf.transform(X_test).toarray()

print("TF-IDF feature size:", X_train_vec.shape[1])

# -----------------------------
# Neural Network (MLP)
# -----------------------------
model = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_vec.shape[1],)),
    Dense(32, activation='relu'),
    Dense(NUM_CLASSES, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# -----------------------------
# EarlyStopping callback
# -----------------------------
early_stop = EarlyStopping(
    monitor='val_loss',   # Stop when validation loss stops improving
    patience=5,           # Wait 5 epochs before stopping
    restore_best_weights=True
)

# -----------------------------
# Train with EarlyStopping
# -----------------------------
model.fit(
    X_train_vec,
    y_train_enc,
    validation_data=(X_val_vec, y_val_enc),
    epochs=100,
    batch_size=16,
    verbose=1,
    callbacks=[early_stop]
)

# -----------------------------
# Evaluate on test set
# -----------------------------
test_preds = model.predict(X_test_vec)
test_preds = np.argmax(test_preds, axis=1)
decoded_preds = label_encoder.inverse_transform(test_preds)

print("\nTest Classification Report:")
print(classification_report(y_test, decoded_preds))

TF-IDF feature size: 2000


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_42 (Dense)                │ (None, 32)             │        64,032 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_43 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_44 (Dense)                │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 65,253 (254.89 KB)

 Trainable params: 65,253 (254.89 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.2872 - loss: 1.5663 - val_accuracy: 0.3559 - val_loss: 1.4453
Epoch 2/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4267 - loss: 1.3520 - val_accuracy: 0.3950 - val_loss: 1.3411
Epoch 3/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5802 - loss: 1.0957 - val_accuracy: 0.4235 - val_loss: 1.3619
Epoch 4/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6912 - loss: 0.8465 - val_accuracy: 0.4288 - val_loss: 1.4474
Epoch 5/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7894 - loss: 0.6198 - val_accuracy: 0.4110 - val_loss: 1.5799
Epoch 6/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8580 - loss: 0.4748 - val_accuracy: 0.4199 - val_loss: 1.7275
Epoch 7/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9012 - loss: 0.3649 - val_accuracy: 0.4306 - val_loss: 1.9195
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 

Test Classification Report:
                          pr

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


TFIDF csere Embeddingre

In [32]:
from tensorflow.keras.layers import Embedding, Flatten, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# -----------------------------
# Encode labels
# -----------------------------
label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# Tokenize texts
# -----------------------------
MAX_VOCAB = 5000
MAX_LEN = 100

tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<UNK>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq   = tokenizer.texts_to_sequences(X_val)
X_test_seq  = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post')
X_val_pad   = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding='post')
X_test_pad  = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post')

# -----------------------------
# Neural Network (Embedding → Flatten → Dense)
# -----------------------------
EMBED_DIM = 32

model = Sequential([
    Embedding(input_dim=MAX_VOCAB, output_dim=EMBED_DIM, input_length=MAX_LEN),
    Flatten(),
    Dense(32, activation='relu'),
    Dense(32, activation='relu'),
    Dense(NUM_CLASSES, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# -----------------------------
# EarlyStopping callback
# -----------------------------
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# -----------------------------
# Train
# -----------------------------
model.fit(
    X_train_pad,
    y_train_enc,
    validation_data=(X_val_pad, y_val_enc),
    epochs=20,
    batch_size=16,
    verbose=1,
    callbacks=[early_stop]
)

# -----------------------------
# Evaluate on test set
# -----------------------------
test_preds = model.predict(X_test_pad)
test_preds = np.argmax(test_preds, axis=1)
decoded_preds = label_encoder.inverse_transform(test_preds)

print("\nTest Classification Report:")
print(classification_report(y_test, decoded_preds))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_45 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_46 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_47 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.3463 - loss: 1.4337 - val_accuracy: 0.4431 - val_loss: 1.3229
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5030 - loss: 1.2260 - val_accuracy: 0.4626 - val_loss: 1.2755
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7561 - loss: 0.7329 - val_accuracy: 0.4840 - val_loss: 1.4157
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9212 - loss: 0.2908 - val_accuracy: 0.4484 - val_loss: 1.6979
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9464 - loss: 0.1868 - val_accuracy: 0.4359 - val_loss: 1.8112
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9647 - loss: 0.1284 - val_accuracy: 0.4502 - val_loss: 1.9868
Epoch 7/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9613 - loss: 0.1152 - val_accuracy: 0.4004 - val_loss: 1.9158
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step

Test Classification Report:
                          precision 

LSTM kipróbálása

In [34]:
from tensorflow.keras.layers import LSTM
label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# Tokenize texts
# -----------------------------
MAX_VOCAB = 5000
MAX_LEN = 100

tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<UNK>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq   = tokenizer.texts_to_sequences(X_val)
X_test_seq  = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post')
X_val_pad   = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding='post')
X_test_pad  = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post')

# -----------------------------
# Neural Network (Embedding → LSTM → Dense)
# -----------------------------
EMBED_DIM = 32
LSTM_UNITS = 16

model = Sequential([
    Embedding(input_dim=MAX_VOCAB, output_dim=EMBED_DIM, input_length=MAX_LEN),
    LSTM(LSTM_UNITS),
    Dense(32, activation='relu'),
    Dense(NUM_CLASSES, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# -----------------------------
# EarlyStopping callback
# -----------------------------
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# -----------------------------
# Train
# -----------------------------
model.fit(
    X_train_pad,
    y_train_enc,
    validation_data=(X_val_pad, y_val_enc),
    epochs=20,
    batch_size=16,
    verbose=1,
    callbacks=[early_stop]
)

# -----------------------------
# Evaluate on test set
# -----------------------------
test_preds = model.predict(X_test_pad)
test_preds = np.argmax(test_preds, axis=1)
decoded_preds = label_encoder.inverse_transform(test_preds)

print("\nTest Classification Report:")
print(classification_report(y_test, decoded_preds))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_9 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_50 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_51 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 10s 33ms/step - accuracy: 0.3002 - loss: 1.5088 - val_accuracy: 0.3523 - val_loss: 1.4049
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 6s 33ms/step - accuracy: 0.3415 - loss: 1.3835 - val_accuracy: 0.4021 - val_loss: 1.3512
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 6s 34ms/step - accuracy: 0.3949 - loss: 1.3618 - val_accuracy: 0.4306 - val_loss: 1.3254
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 11s 37ms/step - accuracy: 0.4467 - loss: 1.2884 - val_accuracy: 0.4217 - val_loss: 1.2987
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - accuracy: 0.5011 - loss: 1.1949 - val_accuracy: 0.4555 - val_loss: 1.2988
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 7s 37ms/step - accuracy: 0.5957 - loss: 1.0520 - val_accuracy: 0.4448 - val_loss: 1.3712
Epoch 7/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 5s 31ms/step - accuracy: 0.6517 - loss: 0.9494 - val_accuracy: 0.4217 - val_loss: 1.4484
Epoch 8/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 7s 37ms/step - accuracy: 0.6873 - loss: 0.8793 - val_ac

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


GRU kipróbálása (elvileg jó kis adathalmazokon)

In [35]:
from tensorflow.keras.layers import GRU
label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# Tokenize texts
# -----------------------------
MAX_VOCAB = 5000
MAX_LEN = 100

tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<UNK>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq   = tokenizer.texts_to_sequences(X_val)
X_test_seq  = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post')
X_val_pad   = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding='post')
X_test_pad  = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post')

# -----------------------------
# Neural Network (Embedding → LSTM → Dense)
# -----------------------------
EMBED_DIM = 32

model = Sequential([
    Embedding(input_dim=MAX_VOCAB, output_dim=EMBED_DIM, input_length=MAX_LEN),
    GRU(32),
    Dense(32, activation='relu'),
    Dense(NUM_CLASSES, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# -----------------------------
# EarlyStopping callback
# -----------------------------
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# -----------------------------
# Train
# -----------------------------
model.fit(
    X_train_pad,
    y_train_enc,
    validation_data=(X_val_pad, y_val_enc),
    epochs=20,
    batch_size=16,
    verbose=1,
    callbacks=[early_stop]
)

# -----------------------------
# Evaluate on test set
# -----------------------------
test_preds = model.predict(X_test_pad)
test_preds = np.argmax(test_preds, axis=1)
decoded_preds = label_encoder.inverse_transform(test_preds)

print("\nTest Classification Report:")
print(classification_report(y_test, decoded_preds))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_20"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_10 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_52 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_53 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 12s 50ms/step - accuracy: 0.2836 - loss: 1.5000 - val_accuracy: 0.3167 - val_loss: 1.4163
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 10s 48ms/step - accuracy: 0.3461 - loss: 1.4164 - val_accuracy: 0.4306 - val_loss: 1.3221
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 7s 42ms/step - accuracy: 0.4462 - loss: 1.3107 - val_accuracy: 0.4342 - val_loss: 1.2891
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 8s 48ms/step - accuracy: 0.5382 - loss: 1.1397 - val_accuracy: 0.4253 - val_loss: 1.3380
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 10s 48ms/step - accuracy: 0.6201 - loss: 0.9683 - val_accuracy: 0.4199 - val_loss: 1.4348
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 7s 42ms/step - accuracy: 0.6581 - loss: 0.8559 - val_accuracy: 0.4164 - val_loss: 1.5764
Epoch 7/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 8s 48ms/step - accuracy: 0.7493 - loss: 0.7236 - val_accuracy: 0.4128 - val_loss: 1.7705
Epoch 8/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 10s 48ms/step - accuracy: 0.7707 - loss: 0.6486 - val_

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Dropout bevezetése

In [37]:
from tensorflow.keras.layers import Dropout

label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# Tokenize texts
# -----------------------------
MAX_VOCAB = 5000
MAX_LEN = 100

tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<UNK>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq   = tokenizer.texts_to_sequences(X_val)
X_test_seq  = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post')
X_val_pad   = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding='post')
X_test_pad  = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post')

# -----------------------------
# Neural Network (Embedding → GRU → Dense) with Dropout
# -----------------------------
EMBED_DIM = 32
GRU_UNITS = 32
DROPOUT_RATE = 0.3  # 30% dropout

model = Sequential([
    Embedding(input_dim=MAX_VOCAB, output_dim=EMBED_DIM, input_length=MAX_LEN),
    GRU(GRU_UNITS),
    Dense(32, activation='relu'),
    Dropout(DROPOUT_RATE),
    Dense(NUM_CLASSES, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# -----------------------------
# EarlyStopping callback
# -----------------------------
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# -----------------------------
# Train
# -----------------------------
model.fit(
    X_train_pad,
    y_train_enc,
    validation_data=(X_val_pad, y_val_enc),
    epochs=20,
    batch_size=16,
    verbose=1,
    callbacks=[early_stop]
)

# -----------------------------
# Evaluate on test set
# -----------------------------
test_preds = model.predict(X_test_pad)
test_preds = np.argmax(test_preds, axis=1)
decoded_preds = label_encoder.inverse_transform(test_preds)

print("\nTest Classification Report:")
print(classification_report(y_test, decoded_preds))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_22"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_12 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_2 (GRU)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_56 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_57 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 15s 53ms/step - accuracy: 0.3032 - loss: 1.5469 - val_accuracy: 0.3221 - val_loss: 1.4361
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 9s 49ms/step - accuracy: 0.3049 - loss: 1.4602 - val_accuracy: 0.3505 - val_loss: 1.3805
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 8s 43ms/step - accuracy: 0.3567 - loss: 1.3897 - val_accuracy: 0.4395 - val_loss: 1.3008
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 9s 49ms/step - accuracy: 0.4680 - loss: 1.2299 - val_accuracy: 0.4484 - val_loss: 1.3022
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 11s 51ms/step - accuracy: 0.5532 - loss: 1.0641 - val_accuracy: 0.4110 - val_loss: 1.3688
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 8s 47ms/step - accuracy: 0.6334 - loss: 0.9619 - val_accuracy: 0.4288 - val_loss: 1.5428
Epoch 7/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 9s 48ms/step - accuracy: 0.6746 - loss: 0.8564 - val_accuracy: 0.4199 - val_loss: 1.6445
Epoch 8/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 9s 53ms/step - accuracy: 0.7369 - loss: 0.7615 - val_ac

Regularizáció bevezetése

In [42]:
from tensorflow.keras.regularizers import l2

label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# Tokenize texts
# -----------------------------
MAX_VOCAB = 5000
MAX_LEN = 100

tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<UNK>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq   = tokenizer.texts_to_sequences(X_val)
X_test_seq  = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post')
X_val_pad   = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding='post')
X_test_pad  = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post')

# -----------------------------
# Neural Network (Embedding → GRU → Dense) with Dropout
# -----------------------------
EMBED_DIM = 32
GRU_UNITS = 32
DROPOUT_RATE = 0.3  # 30% dropout
L2_REG = 1e-4

model = Sequential([
    Embedding(input_dim=MAX_VOCAB, output_dim=EMBED_DIM, input_length=MAX_LEN),
    GRU(GRU_UNITS),
    Dense(32, activation='relu', kernel_regularizer=l2(L2_REG)),
    Dropout(DROPOUT_RATE),
    Dense(NUM_CLASSES, activation='softmax', kernel_regularizer=l2(L2_REG))
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# -----------------------------
# EarlyStopping callback
# -----------------------------
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# -----------------------------
# Train
# -----------------------------
model.fit(
    X_train_pad,
    y_train_enc,
    validation_data=(X_val_pad, y_val_enc),
    epochs=20,
    batch_size=16,
    verbose=1,
    callbacks=[early_stop]
)

# -----------------------------
# Evaluate on test set
# -----------------------------
test_preds = model.predict(X_test_pad)
test_preds = np.argmax(test_preds, axis=1)
decoded_preds = label_encoder.inverse_transform(test_preds)

print("\nTest Classification Report:")
print(classification_report(y_test, decoded_preds))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_27"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_17 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_7 (GRU)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_66 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_67 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 22s 87ms/step - accuracy: 0.2992 - loss: 1.5099 - val_accuracy: 0.3327 - val_loss: 1.4205
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 14s 49ms/step - accuracy: 0.3431 - loss: 1.4282 - val_accuracy: 0.3772 - val_loss: 1.3647
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 9s 43ms/step - accuracy: 0.3911 - loss: 1.3686 - val_accuracy: 0.4359 - val_loss: 1.3046
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 9s 51ms/step - accuracy: 0.4516 - loss: 1.2512 - val_accuracy: 0.4359 - val_loss: 1.2814
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 10s 49ms/step - accuracy: 0.5571 - loss: 1.0946 - val_accuracy: 0.4448 - val_loss: 1.3141
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 8s 45ms/step - accuracy: 0.6130 - loss: 0.9823 - val_accuracy: 0.4324 - val_loss: 1.4261
Epoch 7/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 10s 44ms/step - accuracy: 0.6855 - loss: 0.8627 - val_accuracy: 0.3879 - val_loss: 1.6795
Epoch 8/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 9s 48ms/step - accuracy: 0.7283 - loss: 0.7683 - val_

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [43]:
from tensorflow.keras.layers import Dropout

label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# Tokenize texts
# -----------------------------
MAX_VOCAB = 10000
MAX_LEN = 100

tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<UNK>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq   = tokenizer.texts_to_sequences(X_val)
X_test_seq  = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post')
X_val_pad   = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding='post')
X_test_pad  = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post')

# -----------------------------
# Neural Network (Embedding → GRU → Dense) with Dropout
# -----------------------------
EMBED_DIM = 32
GRU_UNITS = 32
DROPOUT_RATE = 0.3  # 30% dropout

model = Sequential([
    Embedding(input_dim=MAX_VOCAB, output_dim=EMBED_DIM, input_length=MAX_LEN),
    GRU(GRU_UNITS),
    Dense(32, activation='relu'),
    Dropout(DROPOUT_RATE),
    Dense(NUM_CLASSES, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# -----------------------------
# EarlyStopping callback
# -----------------------------
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# -----------------------------
# Train
# -----------------------------
model.fit(
    X_train_pad,
    y_train_enc,
    validation_data=(X_val_pad, y_val_enc),
    epochs=20,
    batch_size=16,
    verbose=1,
    callbacks=[early_stop]
)

# -----------------------------
# Evaluate on test set
# -----------------------------
test_preds = model.predict(X_test_pad)
test_preds = np.argmax(test_preds, axis=1)
decoded_preds = label_encoder.inverse_transform(test_preds)

print("\nTest Classification Report:")
print(classification_report(y_test, decoded_preds))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_28"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_18 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_8 (GRU)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_68 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_69 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 12s 54ms/step - accuracy: 0.3180 - loss: 1.5166 - val_accuracy: 0.3523 - val_loss: 1.4149
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 8s 43ms/step - accuracy: 0.3356 - loss: 1.4564 - val_accuracy: 0.3577 - val_loss: 1.3913
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 9s 49ms/step - accuracy: 0.3408 - loss: 1.3893 - val_accuracy: 0.3719 - val_loss: 1.3614
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 10s 49ms/step - accuracy: 0.4558 - loss: 1.2436 - val_accuracy: 0.4395 - val_loss: 1.2712
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 8s 46ms/step - accuracy: 0.5694 - loss: 1.0588 - val_accuracy: 0.4253 - val_loss: 1.3525
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 8s 47ms/step - accuracy: 0.6594 - loss: 0.8902 - val_accuracy: 0.4359 - val_loss: 1.5100
Epoch 7/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 9s 50ms/step - accuracy: 0.7240 - loss: 0.7487 - val_accuracy: 0.4057 - val_loss: 1.7817
Epoch 8/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 8s 45ms/step - accuracy: 0.7891 - loss: 0.6432 - val_ac

In [44]:
from tensorflow.keras.regularizers import l2

label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# Tokenize texts
# -----------------------------
MAX_VOCAB = 10000
MAX_LEN = 100

tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<UNK>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq   = tokenizer.texts_to_sequences(X_val)
X_test_seq  = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post')
X_val_pad   = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding='post')
X_test_pad  = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post')

# -----------------------------
# Neural Network (Embedding → GRU → Dense) with Dropout
# -----------------------------
EMBED_DIM = 32
GRU_UNITS = 32
DROPOUT_RATE = 0.3  # 30% dropout
L2_REG = 1e-4

model = Sequential([
    Embedding(input_dim=MAX_VOCAB, output_dim=EMBED_DIM, input_length=MAX_LEN),
    GRU(GRU_UNITS),
    Dense(32, activation='relu', kernel_regularizer=l2(L2_REG)),
    Dropout(DROPOUT_RATE),
    Dense(NUM_CLASSES, activation='softmax', kernel_regularizer=l2(L2_REG))
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# -----------------------------
# EarlyStopping callback
# -----------------------------
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# -----------------------------
# Train
# -----------------------------
model.fit(
    X_train_pad,
    y_train_enc,
    validation_data=(X_val_pad, y_val_enc),
    epochs=20,
    batch_size=16,
    verbose=1,
    callbacks=[early_stop]
)

# -----------------------------
# Evaluate on test set
# -----------------------------
test_preds = model.predict(X_test_pad)
test_preds = np.argmax(test_preds, axis=1)
decoded_preds = label_encoder.inverse_transform(test_preds)

print("\nTest Classification Report:")
print(classification_report(y_test, decoded_preds))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_29"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_19 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_9 (GRU)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_70 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_71 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 13s 52ms/step - accuracy: 0.2749 - loss: 1.5531 - val_accuracy: 0.3292 - val_loss: 1.4223
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 8s 43ms/step - accuracy: 0.3100 - loss: 1.4471 - val_accuracy: 0.4199 - val_loss: 1.3564
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 9s 49ms/step - accuracy: 0.4259 - loss: 1.3351 - val_accuracy: 0.4377 - val_loss: 1.3272
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 9s 50ms/step - accuracy: 0.5220 - loss: 1.1324 - val_accuracy: 0.4448 - val_loss: 1.3452
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 8s 43ms/step - accuracy: 0.6358 - loss: 0.9155 - val_accuracy: 0.4431 - val_loss: 1.5004
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 9s 49ms/step - accuracy: 0.6989 - loss: 0.8171 - val_accuracy: 0.4199 - val_loss: 1.5672
Epoch 7/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 9s 50ms/step - accuracy: 0.7321 - loss: 0.7151 - val_accuracy: 0.4164 - val_loss: 1.8834
Epoch 8/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 9s 44ms/step - accuracy: 0.7953 - loss: 0.5962 - val_acc

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Visszatértem sima neurális hálókhoz, az LSTM és a GRU nem teljesítettek jól a tesztjeim alatt.

Különböző perceptron számok és több layer tesztelése


In [47]:
from tensorflow.keras.layers import Embedding, Flatten, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# -----------------------------
# Encode labels
# -----------------------------
label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# Tokenize texts
# -----------------------------
MAX_VOCAB = 5000
MAX_LEN = 100

tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<UNK>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq   = tokenizer.texts_to_sequences(X_val)
X_test_seq  = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post')
X_val_pad   = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding='post')
X_test_pad  = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post')

# -----------------------------
# Neural Network (Embedding → Flatten → Dense)
# -----------------------------
EMBED_DIM = 32

model = Sequential([
    Embedding(input_dim=MAX_VOCAB, output_dim=EMBED_DIM, input_length=MAX_LEN),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(NUM_CLASSES, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# -----------------------------
# EarlyStopping callback
# -----------------------------
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# -----------------------------
# Train
# -----------------------------
model.fit(
    X_train_pad,
    y_train_enc,
    validation_data=(X_val_pad, y_val_enc),
    epochs=20,
    batch_size=16,
    verbose=1,
    callbacks=[early_stop]
)

# -----------------------------
# Evaluate on test set
# -----------------------------
test_preds = model.predict(X_test_pad)
test_preds = np.argmax(test_preds, axis=1)
decoded_preds = label_encoder.inverse_transform(test_preds)

print("\nTest Classification Report:")
print(classification_report(y_test, decoded_preds))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_32"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_22 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_78 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_79 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_80 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - accuracy: 0.3767 - loss: 1.4082 - val_accuracy: 0.4359 - val_loss: 1.3115
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.5384 - loss: 1.1928 - val_accuracy: 0.4466 - val_loss: 1.3663
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.7699 - loss: 0.6587 - val_accuracy: 0.4573 - val_loss: 1.4658
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step - accuracy: 0.9173 - loss: 0.2889 - val_accuracy: 0.4448 - val_loss: 1.6862
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.9513 - loss: 0.1710 - val_accuracy: 0.3879 - val_loss: 1.9251
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.9573 - loss: 0.1308 - val_accuracy: 0.4288 - val_loss: 1.8973
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step

Test Classification Report:
                          precision    recall  f1-score   support

1-Nagyon nehezen érthető       0.00      0.00      0.00        20
       2-Nehezen érthető

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [48]:
from tensorflow.keras.layers import Embedding, Flatten, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# -----------------------------
# Encode labels
# -----------------------------
label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# Tokenize texts
# -----------------------------
MAX_VOCAB = 5000
MAX_LEN = 100

tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<UNK>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq   = tokenizer.texts_to_sequences(X_val)
X_test_seq  = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post')
X_val_pad   = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding='post')
X_test_pad  = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post')

# -----------------------------
# Neural Network (Embedding → Flatten → Dense)
# -----------------------------
EMBED_DIM = 32

model = Sequential([
    Embedding(input_dim=MAX_VOCAB, output_dim=EMBED_DIM, input_length=MAX_LEN),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(32, activation='relu'),
    Dense(NUM_CLASSES, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# -----------------------------
# EarlyStopping callback
# -----------------------------
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# -----------------------------
# Train
# -----------------------------
model.fit(
    X_train_pad,
    y_train_enc,
    validation_data=(X_val_pad, y_val_enc),
    epochs=20,
    batch_size=16,
    verbose=1,
    callbacks=[early_stop]
)

# -----------------------------
# Evaluate on test set
# -----------------------------
test_preds = model.predict(X_test_pad)
test_preds = np.argmax(test_preds, axis=1)
decoded_preds = label_encoder.inverse_transform(test_preds)

print("\nTest Classification Report:")
print(classification_report(y_test, decoded_preds))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_33"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_23 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_81 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_82 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_83 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_84 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.3655 - loss: 1.4199 - val_accuracy: 0.4306 - val_loss: 1.3041
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.5146 - loss: 1.1950 - val_accuracy: 0.4644 - val_loss: 1.2677
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.8190 - loss: 0.5738 - val_accuracy: 0.4448 - val_loss: 1.5152
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.9263 - loss: 0.2449 - val_accuracy: 0.4502 - val_loss: 1.8512
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9532 - loss: 0.1554 - val_accuracy: 0.4181 - val_loss: 1.9845
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9634 - loss: 0.1049 - val_accuracy: 0.4484 - val_loss: 2.0407
Epoch 7/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9643 - loss: 0.0913 - val_accuracy: 0.4395 - val_loss: 2.2004
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step

Test Classification Report:
                          precisi

Innentől rengeteg kombinációban kipróbáltam különböző hálókat különböző dropout paraméterekkel, regularizácioval, embedding mérettel és vocabulary mérettel


In [54]:
label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# Tokenize texts
# -----------------------------
MAX_VOCAB = 5000
MAX_LEN = 100

tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<UNK>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq   = tokenizer.texts_to_sequences(X_val)
X_test_seq  = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post')
X_val_pad   = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding='post')
X_test_pad  = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post')

# -----------------------------
# Neural Network (Embedding → Flatten → Dense) with Dropout
# -----------------------------
EMBED_DIM = 32
DROPOUT_RATE = 0.2
for DROPOUT_RATE in [0.2, 0.3, 0.4]:
  model = Sequential([
      Embedding(input_dim=MAX_VOCAB, output_dim=EMBED_DIM, input_length=MAX_LEN),
      Flatten(),
      Dense(64, activation='relu'),
      Dropout(DROPOUT_RATE),
      Dense(32, activation='relu'),
      Dropout(DROPOUT_RATE),
      Dense(32, activation='relu'),
      Dropout(DROPOUT_RATE),
      Dense(NUM_CLASSES, activation='softmax')
  ])

  model.compile(
      optimizer='adam',
      loss='categorical_crossentropy',
      metrics=['accuracy']
  )

  model.summary()

  # -----------------------------
  # EarlyStopping callback
  # -----------------------------
  early_stop = EarlyStopping(
      monitor='val_loss',
      patience=5,
      restore_best_weights=True
  )

  # -----------------------------
  # Train
  # -----------------------------
  model.fit(
      X_train_pad,
      y_train_enc,
      validation_data=(X_val_pad, y_val_enc),
      epochs=20,
      batch_size=16,
      verbose=1,
      callbacks=[early_stop]
  )

  # -----------------------------
  # Evaluate on test set
  # -----------------------------
  test_preds = model.predict(X_test_pad)
  test_preds = np.argmax(test_preds, axis=1)
  decoded_preds = label_encoder.inverse_transform(test_preds)

  print("\nTest Classification Report:")
  print(classification_report(y_test, decoded_preds))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_39"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_29 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_13 (Flatten)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_105 (Dense)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_27 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_106 (Dense)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_28 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_107 (Dense)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_29 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_108 (Dense)               │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

...

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.3317 - loss: 1.4741 - val_accuracy: 0.4164 - val_loss: 1.4029
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.3960 - loss: 1.3883 - val_accuracy: 0.4484 - val_loss: 1.3005
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.4628 - loss: 1.2496 - val_accuracy: 0.4306 - val_loss: 1.2915
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5873 - loss: 1.0252 - val_accuracy: 0.4715 - val_loss: 1.3697
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.6821 - loss: 0.7912 - val_accuracy: 0.4662 - val_loss: 1.6480
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.7694 - loss: 0.5970 - val_accuracy: 0.4324 - val_loss: 1.9609
Epoch 7/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8375 - loss: 0.4639 - val_accuracy: 0.4431 - val_loss: 2.3873
Epoch 8/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8882 - loss: 0.3467 - val_accuracy: 0

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Bag of Word kipróbálása

In [55]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
import numpy as np

# -----------------------------
# Encode labels
# -----------------------------
label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# Bag-of-Words vectorization
# -----------------------------
MAX_FEATURES = 5000
vectorizer = CountVectorizer(max_features=MAX_FEATURES, ngram_range=(1,2))

X_train_bow = vectorizer.fit_transform(X_train).toarray()
X_val_bow   = vectorizer.transform(X_val).toarray()
X_test_bow  = vectorizer.transform(X_test).toarray()

print("BoW feature size:", X_train_bow.shape[1])

# -----------------------------
# Neural Network (Dense + Dropout)
# -----------------------------
for DROPOUT_RATE in [0.2, 0.3, 0.4]:
    model = Sequential([
        Dense(64, activation='relu', input_shape=(X_train_bow.shape[1],)),
        Dropout(DROPOUT_RATE),
        Dense(32, activation='relu'),
        Dropout(DROPOUT_RATE),
        Dense(32, activation='relu'),
        Dropout(DROPOUT_RATE),
        Dense(NUM_CLASSES, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    model.summary()

    # -----------------------------
    # EarlyStopping callback
    # -----------------------------
    early_stop = EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    )

    # -----------------------------
    # Train
    # -----------------------------
    model.fit(
        X_train_bow,
        y_train_enc,
        validation_data=(X_val_bow, y_val_enc),
        epochs=20,
        batch_size=16,
        verbose=1,
        callbacks=[early_stop]
    )

    # -----------------------------
    # Evaluate on test set
    # -----------------------------
    test_preds = model.predict(X_test_bow)
    test_preds = np.argmax(test_preds, axis=1)
    decoded_preds = label_encoder.inverse_transform(test_preds)

    print("\nTest Classification Report (Dropout={:.1f}):".format(DROPOUT_RATE))
    print(classification_report(y_test, decoded_preds))



BoW feature size: 5000


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_42"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_117 (Dense)               │ (None, 64)             │       320,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_36 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_118 (Dense)               │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_37 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_119 (Dense)               │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_38 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_120 (Dense)               │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 323,365 (1.23 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_125 (Dense)               │ (None, 64)             │       320,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_42 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_126 (Dense)               │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_43 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_127 (Dense)               │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_44 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_128 (Dense)               │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 323,365 (1.23 MB)

 Trainable params: 323,365 (1.23 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.2768 - loss: 1.5745 - val_accuracy: 0.3701 - val_loss: 1.4716
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.3433 - loss: 1.4609 - val_accuracy: 0.3879 - val_loss: 1.3804
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.4425 - loss: 1.3088 - val_accuracy: 0.4591 - val_loss: 1.3037
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.4918 - loss: 1.1977 - val_accuracy: 0.4431 - val_loss: 1.2679
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5556 - loss: 1.0764 - val_accuracy: 0.4715 - val_loss: 1.2987
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6551 - loss: 0.8909 - val_accuracy: 0.4591 - val_loss: 1.3415
Epoch 7/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6995 - loss: 0.8139 - val_accuracy: 0.4431 - val_loss: 1.4234
Epoch 8/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7460 - loss: 0.7077 - val_accuracy: 0

In [57]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
import numpy as np

# -----------------------------
# Encode labels
# -----------------------------
label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# Bag-of-Words vectorization
# -----------------------------
MAX_FEATURES = 10000
vectorizer = CountVectorizer(max_features=MAX_FEATURES, ngram_range=(1,2))

X_train_bow = vectorizer.fit_transform(X_train).toarray()
X_val_bow   = vectorizer.transform(X_val).toarray()
X_test_bow  = vectorizer.transform(X_test).toarray()

print("BoW feature size:", X_train_bow.shape[1])

# -----------------------------
# Neural Network (Dense + Dropout)
# -----------------------------
for DROPOUT_RATE in [0.2, 0.3, 0.4]:
    model = Sequential([
        Dense(64, activation='relu', input_shape=(X_train_bow.shape[1],)),
        Dropout(DROPOUT_RATE),
        Dense(32, activation='relu'),
        Dropout(DROPOUT_RATE),
        Dense(32, activation='relu'),
        Dropout(DROPOUT_RATE),
        Dense(NUM_CLASSES, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    model.summary()

    # -----------------------------
    # EarlyStopping callback
    # -----------------------------
    early_stop = EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    )

    # -----------------------------
    # Train
    # -----------------------------
    model.fit(
        X_train_bow,
        y_train_enc,
        validation_data=(X_val_bow, y_val_enc),
        epochs=20,
        batch_size=16,
        verbose=1,
        callbacks=[early_stop]
    )

    # -----------------------------
    # Evaluate on test set
    # -----------------------------
    test_preds = model.predict(X_test_bow)
    test_preds = np.argmax(test_preds, axis=1)
    decoded_preds = label_encoder.inverse_transform(test_preds)

    print("\nTest Classification Report (Dropout={:.1f}):".format(DROPOUT_RATE))
    print(classification_report(y_test, decoded_preds))


BoW feature size: 10000


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_48"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_141 (Dense)               │ (None, 64)             │       640,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_54 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_142 (Dense)               │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_55 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_143 (Dense)               │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_56 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_144 (Dense)               │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 643,365 (2.45 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_149 (Dense)               │ (None, 64)             │       640,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_60 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_150 (Dense)               │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_61 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_151 (Dense)               │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_62 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_152 (Dense)               │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 643,365 (2.45 MB)

 Trainable params: 643,365 (2.45 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.2713 - loss: 1.5897 - val_accuracy: 0.4181 - val_loss: 1.4567
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.4006 - loss: 1.4297 - val_accuracy: 0.4431 - val_loss: 1.3300
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.4830 - loss: 1.2760 - val_accuracy: 0.4537 - val_loss: 1.2689
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.5556 - loss: 1.1300 - val_accuracy: 0.4448 - val_loss: 1.2854
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.6319 - loss: 0.9495 - val_accuracy: 0.4181 - val_loss: 1.3598
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.6795 - loss: 0.8556 - val_accuracy: 0.4431 - val_loss: 1.4599
Epoch 7/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.7456 - loss: 0.6997 - val_accuracy: 0.4324 - val_loss: 1.5077
Epoch 8/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.7604 - loss: 0.6743 - val_accu

In [61]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
import numpy as np

# -----------------------------
# Encode labels
# -----------------------------
label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# Bag-of-Words vectorization
# -----------------------------
for MAX_FEATURES in [5000, 10000, 20000]:
  vectorizer = CountVectorizer(max_features=MAX_FEATURES, ngram_range=(1,2))

  X_train_bow = vectorizer.fit_transform(X_train).toarray()
  X_val_bow   = vectorizer.transform(X_val).toarray()
  X_test_bow  = vectorizer.transform(X_test).toarray()

  print("BoW feature size:", X_train_bow.shape[1])

  # -----------------------------
  # Neural Network (Dense, no Dropout)
  # -----------------------------
  model = Sequential([
      Dense(64, activation='relu', input_shape=(X_train_bow.shape[1],)),
      Dense(32, activation='relu'),
      Dense(32, activation='relu'),
      Dense(NUM_CLASSES, activation='softmax')
  ])

  model.compile(
      optimizer='adam',
      loss='categorical_crossentropy',
      metrics=['accuracy']
  )

  model.summary()

  # -----------------------------
  # EarlyStopping callback
  # -----------------------------
  early_stop = EarlyStopping(
      monitor='val_loss',
      patience=5,
      restore_best_weights=True
  )

  # -----------------------------
  # Train
  # -----------------------------
  model.fit(
      X_train_bow,
      y_train_enc,
      validation_data=(X_val_bow, y_val_enc),
      epochs=20,
      batch_size=16,
      verbose=1,
      callbacks=[early_stop]
  )

  # -----------------------------
  # Evaluate on test set
  # -----------------------------
  test_preds = model.predict(X_test_bow)
  test_preds = np.argmax(test_preds, axis=1)
  decoded_preds = label_encoder.inverse_transform(test_preds)

  print("\nTest Classification Report:")
  print(classification_report(y_test, decoded_preds))


BoW feature size: 5000


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_57"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_174 (Dense)               │ (None, 64)             │       320,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_175 (Dense)               │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_176 (Dense)               │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_177 (Dense)               │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 323,365 (1.23 MB)

...

 Total params: 1,283,365 (4.90 MB)

 Trainable params: 1,283,365 (4.90 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step - accuracy: 0.3286 - loss: 1.5259 - val_accuracy: 0.4253 - val_loss: 1.2990
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - accuracy: 0.6790 - loss: 0.9040 - val_accuracy: 0.4253 - val_loss: 1.4009
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.8830 - loss: 0.3961 - val_accuracy: 0.4484 - val_loss: 1.8762
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.9302 - loss: 0.2245 - val_accuracy: 0.4110 - val_loss: 2.0852
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 6s 21ms/step - accuracy: 0.9499 - loss: 0.1539 - val_accuracy: 0.4217 - val_loss: 2.2421
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.9514 - loss: 0.1127 - val_accuracy: 0.4075 - val_loss: 2.5532
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step

Test Classification Report:
                          precision    recall  f1-score   support

1-Nagyon nehezen érthető       0.00      0.00      0.00        20
       2-Nehezen érthető

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Visszatérés embeddingre

In [62]:
from tensorflow.keras.layers import Embedding, Flatten, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# -----------------------------
# Encode labels
# -----------------------------
label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# Tokenize texts
# -----------------------------
MAX_VOCAB = 5000
MAX_LEN = 100

tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<UNK>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq   = tokenizer.texts_to_sequences(X_val)
X_test_seq  = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post')
X_val_pad   = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding='post')
X_test_pad  = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post')

# -----------------------------
# Neural Network (Embedding → Flatten → Dense)
# -----------------------------
for EMBED_DIM in [32, 64, 128]:

  model = Sequential([
      Embedding(input_dim=MAX_VOCAB, output_dim=EMBED_DIM, input_length=MAX_LEN),
      Flatten(),
      Dense(64, activation='relu'),
      Dense(32, activation='relu'),
      Dense(32, activation='relu'),
      Dense(NUM_CLASSES, activation='softmax')
  ])

  model.compile(
      optimizer='adam',
      loss='categorical_crossentropy',
      metrics=['accuracy']
  )

  model.summary()

  # -----------------------------
  # EarlyStopping callback
  # -----------------------------
  early_stop = EarlyStopping(
      monitor='val_loss',
      patience=5,
      restore_best_weights=True
  )

  # -----------------------------
  # Train
  # -----------------------------
  model.fit(
      X_train_pad,
      y_train_enc,
      validation_data=(X_val_pad, y_val_enc),
      epochs=20,
      batch_size=16,
      verbose=1,
      callbacks=[early_stop]
  )

  # -----------------------------
  # Evaluate on test set
  # -----------------------------
  test_preds = model.predict(X_test_pad)
  test_preds = np.argmax(test_preds, axis=1)
  decoded_preds = label_encoder.inverse_transform(test_preds)

  print("\nTest Classification Report:")
  print(classification_report(y_test, decoded_preds))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_60"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_32 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_16 (Flatten)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_186 (Dense)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_187 (Dense)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_188 (Dense)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_189 (Dense)               │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_34 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_18 (Flatten)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_194 (Dense)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_195 (Dense)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_196 (Dense)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_197 (Dense)               │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - accuracy: 0.3744 - loss: 1.4085 - val_accuracy: 0.4146 - val_loss: 1.3036
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.5536 - loss: 1.1312 - val_accuracy: 0.4431 - val_loss: 1.2883
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - accuracy: 0.8728 - loss: 0.4326 - val_accuracy: 0.4502 - val_loss: 1.6153
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.9457 - loss: 0.1960 - val_accuracy: 0.4199 - val_loss: 1.7239
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.9661 - loss: 0.1118 - val_accuracy: 0.4181 - val_loss: 2.1000
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.9682 - loss: 0.0834 - val_accuracy: 0.4128 - val_loss: 2.1892
Epoch 7/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.9663 - loss: 0.0666 - val_accuracy: 0.4235 - val_loss: 2.3763
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step

Test Classification Report:
                          pr

In [63]:
from tensorflow.keras.layers import Embedding, Flatten, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# -----------------------------
# Encode labels
# -----------------------------
label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# Tokenize texts
# -----------------------------
MAX_VOCAB = 5000
MAX_LEN = 100

tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<UNK>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq   = tokenizer.texts_to_sequences(X_val)
X_test_seq  = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post')
X_val_pad   = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding='post')
X_test_pad  = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post')

# -----------------------------
# Neural Network (Embedding → Flatten → Dense)
# -----------------------------
for EMBED_DIM in [32, 64, 128]:

  model = Sequential([
      Embedding(input_dim=MAX_VOCAB, output_dim=EMBED_DIM, input_length=MAX_LEN),
      Flatten(),
      Dense(32, activation='relu'),
      Dense(32, activation='relu'),
      Dense(NUM_CLASSES, activation='softmax')
  ])

  model.compile(
      optimizer='adam',
      loss='categorical_crossentropy',
      metrics=['accuracy']
  )

  model.summary()

  # -----------------------------
  # EarlyStopping callback
  # -----------------------------
  early_stop = EarlyStopping(
      monitor='val_loss',
      patience=5,
      restore_best_weights=True
  )

  # -----------------------------
  # Train
  # -----------------------------
  model.fit(
      X_train_pad,
      y_train_enc,
      validation_data=(X_val_pad, y_val_enc),
      epochs=20,
      batch_size=16,
      verbose=1,
      callbacks=[early_stop]
  )

  # -----------------------------
  # Evaluate on test set
  # -----------------------------
  test_preds = model.predict(X_test_pad)
  test_preds = np.argmax(test_preds, axis=1)
  decoded_preds = label_encoder.inverse_transform(test_preds)

  print("\nTest Classification Report:")
  print(classification_report(y_test, decoded_preds))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_63"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_35 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_19 (Flatten)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_198 (Dense)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_199 (Dense)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_200 (Dense)               │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_37 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_21 (Flatten)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_204 (Dense)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_205 (Dense)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_206 (Dense)               │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.3957 - loss: 1.3769 - val_accuracy: 0.4359 - val_loss: 1.3099
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.6294 - loss: 1.0374 - val_accuracy: 0.4537 - val_loss: 1.2999
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.8903 - loss: 0.3702 - val_accuracy: 0.4395 - val_loss: 1.5518
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.9469 - loss: 0.1820 - val_accuracy: 0.4217 - val_loss: 1.7281
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.9648 - loss: 0.1057 - val_accuracy: 0.4270 - val_loss: 1.6966
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.9643 - loss: 0.0983 - val_accuracy: 0.3950 - val_loss: 1.8699
Epoch 7/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.9632 - loss: 0.0768 - val_accuracy: 0.4093 - val_loss: 2.0834
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step

Test Classification Report:
                          pre

In [64]:
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import numpy as np

# -----------------------------
# Encode labels
# -----------------------------
label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# Tokenize texts
# -----------------------------
MAX_VOCAB = 5000
MAX_LEN = 100

tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<UNK>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq   = tokenizer.texts_to_sequences(X_val)
X_test_seq  = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post')
X_val_pad   = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding='post')
X_test_pad  = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post')

# -----------------------------
# Neural Network (Embedding → Flatten → Dense with Dropout)
# -----------------------------
for EMBED_DIM in [32, 64, 128]:
    for DROPOUT_RATE in [0.2, 0.3, 0.4]:

      model = Sequential([
          Embedding(input_dim=MAX_VOCAB, output_dim=EMBED_DIM, input_length=MAX_LEN),
          Flatten(),
          Dense(32, activation='relu'),
          Dropout(DROPOUT_RATE),
          Dense(32, activation='relu'),
          Dropout(DROPOUT_RATE),
          Dense(NUM_CLASSES, activation='softmax')
      ])

      model.compile(
          optimizer='adam',
          loss='categorical_crossentropy',
          metrics=['accuracy']
      )

      model.summary()

      # -----------------------------
      # EarlyStopping callback
      # -----------------------------
      early_stop = EarlyStopping(
          monitor='val_loss',
          patience=5,
          restore_best_weights=True
      )

      # -----------------------------
      # Train
      # -----------------------------
      model.fit(
          X_train_pad,
          y_train_enc,
          validation_data=(X_val_pad, y_val_enc),
          epochs=20,
          batch_size=16,
          verbose=1,
          callbacks=[early_stop]
      )

      # -----------------------------
      # Evaluate on test set
      # -----------------------------
      test_preds = model.predict(X_test_pad)
      test_preds = np.argmax(test_preds, axis=1)
      decoded_preds = label_encoder.inverse_transform(test_preds)

      print("\nTest Classification Report (Embedding dim={}):".format(EMBED_DIM))
      print(classification_report(y_test, decoded_preds))


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_66"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_38 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_22 (Flatten)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_207 (Dense)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_63 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_208 (Dense)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_64 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_209 (Dense)               │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_46 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_30 (Flatten)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_231 (Dense)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_79 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_232 (Dense)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_80 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_233 (Dense)               │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.3037 - loss: 1.5198 - val_accuracy: 0.4431 - val_loss: 1.3342
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.4245 - loss: 1.3229 - val_accuracy: 0.4555 - val_loss: 1.2748
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.5366 - loss: 1.1129 - val_accuracy: 0.4537 - val_loss: 1.3229
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.7136 - loss: 0.7788 - val_accuracy: 0.4609 - val_loss: 1.3722
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.8432 - loss: 0.4998 - val_accuracy: 0.4573 - val_loss: 1.6365
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.9039 - loss: 0.3360 - val_accuracy: 0.4413 - val_loss: 1.7600
Epoch 7/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.9132 - loss: 0.2871 - val_accuracy: 0.4270 - val_loss: 1.8885
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step

Test Classification Report (Embedding dim=128):
         

In [67]:
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import numpy as np

# -----------------------------
# Encode labels
# -----------------------------
label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# Tokenize texts
# -----------------------------
MAX_VOCAB = 5000
MAX_LEN = 100

tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<UNK>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq   = tokenizer.texts_to_sequences(X_val)
X_test_seq  = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post')
X_val_pad   = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding='post')
X_test_pad  = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post')

# -----------------------------
# Neural Network (Embedding → Flatten → Dense with Dropout)
# -----------------------------
for EMBED_DIM in [32]:
    for DROPOUT_RATE in [0.0, 0.3]:

      model = Sequential([
          Embedding(input_dim=MAX_VOCAB, output_dim=EMBED_DIM, input_length=MAX_LEN),
          Flatten(),
          Dense(32, activation='relu'),
          Dropout(DROPOUT_RATE),
          Dense(32, activation='relu'),
          Dropout(DROPOUT_RATE),
          Dense(NUM_CLASSES, activation='softmax')
      ])

      model.compile(
          optimizer='adam',
          loss='categorical_crossentropy',
          metrics=['accuracy']
      )

      model.summary()

      # -----------------------------
      # EarlyStopping callback
      # -----------------------------
      early_stop = EarlyStopping(
          monitor='val_loss',
          patience=5,
          restore_best_weights=True
      )

      # -----------------------------
      # Train
      # -----------------------------
      model.fit(
          X_train_pad,
          y_train_enc,
          validation_data=(X_val_pad, y_val_enc),
          epochs=20,
          batch_size=16,
          verbose=1,
          callbacks=[early_stop]
      )

      # -----------------------------
      # Evaluate on test set
      # -----------------------------
      test_preds = model.predict(X_test_pad)
      test_preds = np.argmax(test_preds, axis=1)
      decoded_preds = label_encoder.inverse_transform(test_preds)

      print("\nTest Classification Report (Embedding dim={}):".format(EMBED_DIM))
      print(classification_report(y_test, decoded_preds))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_77"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_49 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_33 (Flatten)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_240 (Dense)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_85 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_241 (Dense)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_86 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_242 (Dense)               │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_50 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_34 (Flatten)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_243 (Dense)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_87 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_244 (Dense)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_88 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_245 (Dense)               │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.3385 - loss: 1.4797 - val_accuracy: 0.4146 - val_loss: 1.3292
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.4461 - loss: 1.3283 - val_accuracy: 0.4555 - val_loss: 1.2856
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5917 - loss: 1.1025 - val_accuracy: 0.4431 - val_loss: 1.2804
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.7576 - loss: 0.7090 - val_accuracy: 0.4324 - val_loss: 1.4766
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9013 - loss: 0.3705 - val_accuracy: 0.4110 - val_loss: 1.6887
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9248 - loss: 0.2797 - val_accuracy: 0.4128 - val_loss: 1.8332
Epoch 7/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9443 - loss: 0.2008 - val_accuracy: 0.4075 - val_loss: 2.0456
Epoch 8/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9527 - loss: 0.1560 - val_accuracy: 0.

In [68]:
from tensorflow.keras.layers import Embedding, Flatten, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# -----------------------------
# Encode labels
# -----------------------------
label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# Tokenize texts
# -----------------------------
for MAX_VOCAB in [5000, 10000, 20000]:
  MAX_LEN = 100

  tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<UNK>")
  tokenizer.fit_on_texts(X_train)

  X_train_seq = tokenizer.texts_to_sequences(X_train)
  X_val_seq   = tokenizer.texts_to_sequences(X_val)
  X_test_seq  = tokenizer.texts_to_sequences(X_test)

  X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post')
  X_val_pad   = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding='post')
  X_test_pad  = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post')

  # -----------------------------
  # Neural Network (Embedding → Flatten → Dense)
  # -----------------------------
  for EMBED_DIM in [32]:

    model = Sequential([
        Embedding(input_dim=MAX_VOCAB, output_dim=EMBED_DIM, input_length=MAX_LEN),
        Flatten(),
        Dense(32, activation='relu'),
        Dense(32, activation='relu'),
        Dense(NUM_CLASSES, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    model.summary()

    # -----------------------------
    # EarlyStopping callback
    # -----------------------------
    early_stop = EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    )

    # -----------------------------
    # Train
    # -----------------------------
    model.fit(
        X_train_pad,
        y_train_enc,
        validation_data=(X_val_pad, y_val_enc),
        epochs=20,
        batch_size=16,
        verbose=1,
        callbacks=[early_stop]
    )

    # -----------------------------
    # Evaluate on test set
    # -----------------------------
    test_preds = model.predict(X_test_pad)
    test_preds = np.argmax(test_preds, axis=1)
    decoded_preds = label_encoder.inverse_transform(test_preds)

    print("\nTest Classification Report:")
    print(classification_report(y_test, decoded_preds))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_79"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_51 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_35 (Flatten)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_246 (Dense)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_247 (Dense)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_248 (Dense)               │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_53 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_37 (Flatten)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_252 (Dense)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_253 (Dense)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_254 (Dense)               │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.3678 - loss: 1.4268 - val_accuracy: 0.4520 - val_loss: 1.3205
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5476 - loss: 1.1682 - val_accuracy: 0.4537 - val_loss: 1.3084
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.8398 - loss: 0.5676 - val_accuracy: 0.4448 - val_loss: 1.5796
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9316 - loss: 0.2158 - val_accuracy: 0.4448 - val_loss: 1.7280
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.9555 - loss: 0.1359 - val_accuracy: 0.4502 - val_loss: 1.7649
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9636 - loss: 0.1004 - val_accuracy: 0.4270 - val_loss: 2.1626
Epoch 7/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9675 - loss: 0.0817 - val_accuracy: 0.4110 - val_loss: 2.2307
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step

Test Classification Report:
                          precisi

In [70]:
label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# Tokenize texts
# -----------------------------
for MAX_VOCAB in [5000, 10000, 20000]:
  MAX_LEN = 100

  tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<UNK>")
  tokenizer.fit_on_texts(X_train)

  X_train_seq = tokenizer.texts_to_sequences(X_train)
  X_val_seq   = tokenizer.texts_to_sequences(X_val)
  X_test_seq  = tokenizer.texts_to_sequences(X_test)

  X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post')
  X_val_pad   = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding='post')
  X_test_pad  = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post')

  # -----------------------------
  # Neural Network (Embedding → Flatten → Dense with Dropout)
  # -----------------------------
  for EMBED_DIM in [32]:
      for DROPOUT_RATE in [0.2, 0.3, 0.4]:

        model = Sequential([
            Embedding(input_dim=MAX_VOCAB, output_dim=EMBED_DIM, input_length=MAX_LEN),
            Flatten(),
            Dense(32, activation='relu'),
            Dropout(DROPOUT_RATE),
            Dense(32, activation='relu'),
            Dropout(DROPOUT_RATE),
            Dense(NUM_CLASSES, activation='softmax')
        ])

        model.compile(
            optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

        model.summary()

        # -----------------------------
        # EarlyStopping callback
        # -----------------------------
        early_stop = EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )

        # -----------------------------
        # Train
        # -----------------------------
        model.fit(
            X_train_pad,
            y_train_enc,
            validation_data=(X_val_pad, y_val_enc),
            epochs=20,
            batch_size=16,
            verbose=1,
            callbacks=[early_stop]
        )

        # -----------------------------
        # Evaluate on test set
        # -----------------------------
        test_preds = model.predict(X_test_pad)
        test_preds = np.argmax(test_preds, axis=1)
        decoded_preds = label_encoder.inverse_transform(test_preds)
        print(f"Max vocab:{MAX_VOCAB}")
        print(f"Dropout rate:{DROPOUT_RATE}")
        print("\nTest Classification Report (Embedding dim={}):".format(EMBED_DIM))
        print(classification_report(y_test, decoded_preds))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_83"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_55 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_39 (Flatten)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_258 (Dense)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_91 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_259 (Dense)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_92 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_260 (Dense)               │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

...

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.3234 - loss: 1.4983 - val_accuracy: 0.4128 - val_loss: 1.3535
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.4183 - loss: 1.3523 - val_accuracy: 0.4413 - val_loss: 1.3276
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.5660 - loss: 1.0890 - val_accuracy: 0.4342 - val_loss: 1.3026
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.7615 - loss: 0.6962 - val_accuracy: 0.4306 - val_loss: 1.4598
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.8712 - loss: 0.4191 - val_accuracy: 0.4110 - val_loss: 1.7196
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9167 - loss: 0.2934 - val_accuracy: 0.4110 - val_loss: 1.7602
Epoch 7/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9200 - loss: 0.2668 - val_accuracy: 0.4342 - val_loss: 2.0338
Epoch 8/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9408 - loss: 0.2074 - val_accuracy

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [71]:
label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# Tokenize texts
# -----------------------------
for MAX_VOCAB in [10000]:
  MAX_LEN = 100

  tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<UNK>")
  tokenizer.fit_on_texts(X_train)

  X_train_seq = tokenizer.texts_to_sequences(X_train)
  X_val_seq   = tokenizer.texts_to_sequences(X_val)
  X_test_seq  = tokenizer.texts_to_sequences(X_test)

  X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post')
  X_val_pad   = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding='post')
  X_test_pad  = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post')

  # -----------------------------
  # Neural Network (Embedding → Flatten → Dense with Dropout)
  # -----------------------------
  for EMBED_DIM in [32]:
      for DROPOUT_RATE in [0.4]:

        model = Sequential([
            Embedding(input_dim=MAX_VOCAB, output_dim=EMBED_DIM, input_length=MAX_LEN),
            Flatten(),
            Dense(32, activation='relu'),
            Dropout(DROPOUT_RATE),
            Dense(32, activation='relu'),
            Dropout(DROPOUT_RATE),
            Dense(NUM_CLASSES, activation='softmax')
        ])

        model.compile(
            optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

        model.summary()

        # -----------------------------
        # EarlyStopping callback
        # -----------------------------
        early_stop = EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )

        # -----------------------------
        # Train
        # -----------------------------
        model.fit(
            X_train_pad,
            y_train_enc,
            validation_data=(X_val_pad, y_val_enc),
            epochs=20,
            batch_size=16,
            verbose=1,
            callbacks=[early_stop]
        )

        # -----------------------------
        # Evaluate on test set
        # -----------------------------
        test_preds = model.predict(X_test_pad)
        test_preds = np.argmax(test_preds, axis=1)
        decoded_preds = label_encoder.inverse_transform(test_preds)
        print(f"Max vocab:{MAX_VOCAB}")
        print(f"Dropout rate:{DROPOUT_RATE}")
        print("\nTest Classification Report (Embedding dim={}):".format(EMBED_DIM))
        print(classification_report(y_test, decoded_preds))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_92"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_64 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_48 (Flatten)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_285 (Dense)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_109 (Dropout)           │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_286 (Dense)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_110 (Dropout)           │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_287 (Dense)               │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.3217 - loss: 1.5050 - val_accuracy: 0.4181 - val_loss: 1.3461
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.4117 - loss: 1.3544 - val_accuracy: 0.4555 - val_loss: 1.3072
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5106 - loss: 1.1724 - val_accuracy: 0.4644 - val_loss: 1.2848
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6906 - loss: 0.8198 - val_accuracy: 0.4502 - val_loss: 1.4172
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.8223 - loss: 0.5040 - val_accuracy: 0.4342 - val_loss: 1.6095
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8815 - loss: 0.3651 - val_accuracy: 0.4377 - val_loss: 1.8184
Epoch 7/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9271 - loss: 0.2653 - val_accuracy: 0.4235 - val_loss: 1.8820
Epoch 8/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9392 - loss: 0.2171 - val_accuracy: 0

In [72]:
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import numpy as np

# -----------------------------
# Encode labels
# -----------------------------
label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# Tokenize texts
# -----------------------------
for MAX_VOCAB in [10000]:
    MAX_LEN = 100

    tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<UNK>")
    tokenizer.fit_on_texts(X_train)

    X_train_seq = tokenizer.texts_to_sequences(X_train)
    X_val_seq   = tokenizer.texts_to_sequences(X_val)
    X_test_seq  = tokenizer.texts_to_sequences(X_test)

    X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post')
    X_val_pad   = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding='post')
    X_test_pad  = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post')

    # -----------------------------
    # Neural Network (Embedding → Flatten → Dense with Dropout + L2)
    # -----------------------------
    for EMBED_DIM in [32]:
        for DROPOUT_RATE in [0.4]:
            L2_REG = 1e-4  # L2 regularization factor

            model = Sequential([
                Embedding(input_dim=MAX_VOCAB, output_dim=EMBED_DIM, input_length=MAX_LEN),
                Flatten(),
                Dense(32, activation='relu', kernel_regularizer=l2(L2_REG)),
                Dropout(DROPOUT_RATE),
                Dense(32, activation='relu', kernel_regularizer=l2(L2_REG)),
                Dropout(DROPOUT_RATE),
                Dense(NUM_CLASSES, activation='softmax', kernel_regularizer=l2(L2_REG))
            ])

            model.compile(
                optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy']
            )

            model.summary()

            # -----------------------------
            # EarlyStopping callback
            # -----------------------------
            early_stop = EarlyStopping(
                monitor='val_loss',
                patience=5,
                restore_best_weights=True
            )

            # -----------------------------
            # Train
            # -----------------------------
            model.fit(
                X_train_pad,
                y_train_enc,
                validation_data=(X_val_pad, y_val_enc),
                epochs=20,
                batch_size=16,
                verbose=1,
                callbacks=[early_stop]
            )

            # -----------------------------
            # Evaluate on test set
            # -----------------------------
            test_preds = model.predict(X_test_pad)
            test_preds = np.argmax(test_preds, axis=1)
            decoded_preds = label_encoder.inverse_transform(test_preds)

            print(f"Max vocab: {MAX_VOCAB}")
            print(f"Dropout rate: {DROPOUT_RATE}")
            print("\nTest Classification Report (Embedding dim={}):".format(EMBED_DIM))
            print(classification_report(y_test, decoded_preds))


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_93"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_65 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_49 (Flatten)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_288 (Dense)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_111 (Dropout)           │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_289 (Dense)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_112 (Dropout)           │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_290 (Dense)               │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.2846 - loss: 1.5431 - val_accuracy: 0.4377 - val_loss: 1.3629
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.3971 - loss: 1.3929 - val_accuracy: 0.4484 - val_loss: 1.3063
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.4999 - loss: 1.2077 - val_accuracy: 0.4662 - val_loss: 1.2909
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.6932 - loss: 0.8495 - val_accuracy: 0.4270 - val_loss: 1.4521
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8202 - loss: 0.5703 - val_accuracy: 0.4199 - val_loss: 1.7885
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9004 - loss: 0.3764 - val_accuracy: 0.4502 - val_loss: 1.9724
Epoch 7/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9227 - loss: 0.2859 - val_accuracy: 0.3754 - val_loss: 2.1391
Epoch 8/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9387 - loss: 0.2493 - val_accuracy: 

In [74]:
label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# Tokenize texts
# -----------------------------
for MAX_VOCAB in [10000]:
  MAX_LEN = 100

  tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<UNK>")
  tokenizer.fit_on_texts(X_train)

  X_train_seq = tokenizer.texts_to_sequences(X_train)
  X_val_seq   = tokenizer.texts_to_sequences(X_val)
  X_test_seq  = tokenizer.texts_to_sequences(X_test)

  X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post')
  X_val_pad   = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding='post')
  X_test_pad  = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post')

  # -----------------------------
  # Neural Network (Embedding → Flatten → Dense with Dropout)
  # -----------------------------
  for EMBED_DIM in [32]:
      for DROPOUT_RATE in [0.4]:

        model = Sequential([
            Embedding(input_dim=MAX_VOCAB, output_dim=EMBED_DIM, input_length=MAX_LEN),
            Flatten(),
            Dense(32, activation='relu'),
            Dropout(DROPOUT_RATE),
            Dense(32, activation='relu'),
            Dropout(DROPOUT_RATE),
            Dense(NUM_CLASSES, activation='softmax')
        ])

        model.compile(
            optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

        model.summary()

        # -----------------------------
        # EarlyStopping callback
        # -----------------------------
        early_stop = EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )

        # -----------------------------
        # Train
        # -----------------------------
        model.fit(
            X_train_pad,
            y_train_enc,
            validation_data=(X_val_pad, y_val_enc),
            epochs=20,
            batch_size=8,
            verbose=1,
            callbacks=[early_stop]
        )

        # -----------------------------
        # Evaluate on test set
        # -----------------------------
        test_preds = model.predict(X_test_pad)
        test_preds = np.argmax(test_preds, axis=1)
        decoded_preds = label_encoder.inverse_transform(test_preds)
        print(f"Max vocab:{MAX_VOCAB}")
        print(f"Dropout rate:{DROPOUT_RATE}")
        print("\nTest Classification Report (Embedding dim={}):".format(EMBED_DIM))
        print(classification_report(y_test, decoded_preds))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_95"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_67 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_51 (Flatten)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_294 (Dense)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_115 (Dropout)           │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_295 (Dense)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_116 (Dropout)           │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_296 (Dense)               │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step - accuracy: 0.3316 - loss: 1.4951 - val_accuracy: 0.4342 - val_loss: 1.3365
Epoch 2/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.4347 - loss: 1.3189 - val_accuracy: 0.4662 - val_loss: 1.2607
Epoch 3/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.6310 - loss: 0.9708 - val_accuracy: 0.4395 - val_loss: 1.2899
Epoch 4/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8215 - loss: 0.5534 - val_accuracy: 0.4253 - val_loss: 1.5660
Epoch 5/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9077 - loss: 0.3335 - val_accuracy: 0.4431 - val_loss: 1.8815
Epoch 6/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9187 - loss: 0.2533 - val_accuracy: 0.4164 - val_loss: 1.9347
Epoch 7/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9422 - loss: 0.2061 - val_accuracy: 0.4253 - val_loss: 2.2524
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
Max vocab:10000
Dropout rate:0.4

Test Classification Report (Em

Végső választott modell.
Később a dropout törölve lett, mert úgy jobb eredményt adott.

In [ ]:
#Végső modell tesztelése új adathalmaz felbontásban


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
import numpy as np



# -----------------------------
# Encode labels
# -----------------------------
label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# Tokenize texts
# -----------------------------
MAX_VOCAB = 10000
MAX_LEN = 100

tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<UNK>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq   = tokenizer.texts_to_sequences(X_val)
X_test_seq  = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post')
X_val_pad   = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding='post')
X_test_pad  = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post')

# -----------------------------
# Neural Network (Embedding → Flatten → Dense with Dropout)
# -----------------------------
EMBED_DIM = 32
DROPOUT_RATE = 0.4

model = Sequential([
    Embedding(input_dim=MAX_VOCAB, output_dim=EMBED_DIM, input_length=MAX_LEN),
    Flatten(),
    Dense(32, activation='relu'),
    Dropout(DROPOUT_RATE),
    Dense(32, activation='relu'),
    Dropout(DROPOUT_RATE),
    Dense(NUM_CLASSES, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# -----------------------------
# EarlyStopping callback
# -----------------------------
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# -----------------------------
# Train
# -----------------------------
model.fit(
    X_train_pad,
    y_train_enc,
    validation_data=(X_val_pad, y_val_enc),
    epochs=20,
    batch_size=16,
    verbose=1,
    callbacks=[early_stop]
)

# -----------------------------
# Evaluate on test set
# -----------------------------
test_preds = model.predict(X_test_pad)
test_preds = np.argmax(test_preds, axis=1)
decoded_preds = label_encoder.inverse_transform(test_preds)

print(f"\nMax vocab: {MAX_VOCAB}")
print(f"Dropout rate: {DROPOUT_RATE}")
print("\nTest Classification Report (Embedding dim={}):".format(EMBED_DIM))
print(classification_report(y_test, decoded_preds))


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_101"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_73 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_57 (Flatten)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_312 (Dense)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_127 (Dropout)           │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_313 (Dense)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_128 (Dropout)           │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_314 (Dense)               │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.3316 - loss: 1.4665 - val_accuracy: 0.4306 - val_loss: 1.3511
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.4546 - loss: 1.3468 - val_accuracy: 0.4520 - val_loss: 1.2899
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.5479 - loss: 1.1542 - val_accuracy: 0.4840 - val_loss: 1.2587
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.7279 - loss: 0.7914 - val_accuracy: 0.4502 - val_loss: 1.3480
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.8428 - loss: 0.4801 - val_accuracy: 0.4306 - val_loss: 1.5677
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9091 - loss: 0.3182 - val_accuracy: 0.4413 - val_loss: 1.7961
Epoch 7/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9347 - loss: 0.2548 - val_accuracy: 0.4004 - val_loss: 1.9058
Epoch 8/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9358 - loss: 0.2120 - val_accuracy: 

In [88]:
#Lehetséges megoldás a kiegyenlítetlenségre
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

#Végső modell tesztelése új adathalmaz felbontásban


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
import numpy as np



# -----------------------------
# Encode labels
# -----------------------------
label_encoder = LabelEncoder()
label_encoder.fit(labels)

y_train_enc = label_encoder.transform(y_train)
y_val_enc   = label_encoder.transform(y_val)
y_test_enc  = label_encoder.transform(y_test)

NUM_CLASSES = len(label_encoder.classes_)
y_train_enc = to_categorical(y_train_enc, num_classes=NUM_CLASSES)
y_val_enc   = to_categorical(y_val_enc, num_classes=NUM_CLASSES)

# -----------------------------
# Tokenize texts
# -----------------------------
MAX_VOCAB = 10000
MAX_LEN = 100

tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<UNK>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq   = tokenizer.texts_to_sequences(X_val)
X_test_seq  = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post')
X_val_pad   = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding='post')
X_test_pad  = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post')

# -----------------------------
# Neural Network (Embedding → Flatten → Dense with Dropout)
# -----------------------------
EMBED_DIM = 32
DROPOUT_RATE = 0.4

model = Sequential([
    Embedding(input_dim=MAX_VOCAB, output_dim=EMBED_DIM, input_length=MAX_LEN),
    Flatten(),
    Dense(32, activation='relu'),
    Dropout(DROPOUT_RATE),
    Dense(32, activation='relu'),
    Dropout(DROPOUT_RATE),
    Dense(NUM_CLASSES, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# -----------------------------
# EarlyStopping callback
# -----------------------------
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

y_train_int = label_encoder.transform(y_train)  # already done as y_train_enc before one-hot
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train_int),
    y=y_train_int
)
class_weights_dict = dict(enumerate(class_weights))

# -----------------------------
# Train model with class weights
# -----------------------------
model.fit(
    X_train_pad,
    y_train_enc,
    validation_data=(X_val_pad, y_val_enc),
    epochs=20,
    batch_size=16,
    verbose=1,
    callbacks=[early_stop],
    class_weight=class_weights_dict   # <-- add here
)

# -----------------------------
# Evaluate on test set
# -----------------------------
test_preds = model.predict(X_test_pad)
test_preds = np.argmax(test_preds, axis=1)
decoded_preds = label_encoder.inverse_transform(test_preds)

print(f"\nMax vocab: {MAX_VOCAB}")
print(f"Dropout rate: {DROPOUT_RATE}")
print("\nTest Classification Report (Embedding dim={}):".format(EMBED_DIM))
print(classification_report(y_test, decoded_preds))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_100"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_72 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_56 (Flatten)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_309 (Dense)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_125 (Dropout)           │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_310 (Dense)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_126 (Dropout)           │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_311 (Dense)               │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.3111 - loss: 1.5581 - val_accuracy: 0.3701 - val_loss: 1.4606
Epoch 2/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.3627 - loss: 1.5110 - val_accuracy: 0.3897 - val_loss: 1.3929
Epoch 3/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.4113 - loss: 1.3196 - val_accuracy: 0.3861 - val_loss: 1.3361
Epoch 4/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5616 - loss: 0.9760 - val_accuracy: 0.4199 - val_loss: 1.3736
Epoch 5/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6842 - loss: 0.7033 - val_accuracy: 0.4306 - val_loss: 1.4069
Epoch 6/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8032 - loss: 0.4811 - val_accuracy: 0.4093 - val_loss: 1.5994
Epoch 7/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.8614 - loss: 0.4002 - val_accuracy: 0.4004 - val_loss: 1.7216
Epoch 8/20
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8915 - loss: 0.3578 - val_accuracy: 0

Accuracy romlott, de minden osztályra becsül a modell (végül nem lett használva)